In [1]:
import os

In [2]:
%pwd

'c:\\Users\\akash\\Desktop\\Chicken-Disease-DL-CloudDeployment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\akash\\Desktop\\Chicken-Disease-DL-CloudDeployment'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-03 09:06:25,801: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-03 09:06:25,805: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-03 09:06:25,808: INFO: common: created directory at: artifacts]
[2023-12-03 09:06:25,809: INFO: common: created directory at: artifacts/data_ingestion]


[2023-12-03 09:06:34,368: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25193201
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5501a599862a21f871586d4120df2cee22c05e42875b160af25d9f29f8088112"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A7F4:7C22:33DF51:437FFA:656BF7BB
Accept-Ranges: bytes
Date: Sun, 03 Dec 2023 03:36:32 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10243-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1701574590.698805,VS0,VE3177
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a658fe7f5787b80960a7e50594d9cf7e4b17dec6
Expires: Sun, 03 Dec 2023 03:41:32 GMT
Source-Age: 3

]
